In [2]:
!pip install transformers
!pip install sentencepiece
!pip install nltk
!pip install ipywidgets
!pip install tensorboard

In [582]:
def check (lst1 , lst2):
    if lst1 == lst2:
        return True
    else :
        return False
# lst1 = ['a','b',"\\",'d','e','\\','f','\\','\\']
lst1 = ['g','e','b','\\','t','b']
# lst1 = ['a','b',"\\",'d','\\','\\']
lst=[]
# lst1 = ['a',"\\","\\","\\","\\"]
lst2 = ['g', 'e', 't', 'b']
# lst2=[]
indices= [ i for i,item in enumerate(lst1) if item == '\\']
# print(indices)
for i,item in enumerate(indices):
    
    if i == 0:
        lst.extend(lst1[:item-1])
        
    else:
        lst.extend(lst1[indices[i-1]+1:item-1])
    if len(indices) == i+1:
        lst.extend(lst1[item+1:])

check(lst,lst2)



True

In [4]:
from collections import OrderedDict
 
class LRUCache:
 
    # initialising capacity
    def __init__(self, capacity: int):
        self.cache = OrderedDict()
        self.capacity = capacity
 
    # we return the value of the key
    # that is queried in O(1) and return -1 if we
    # don't find the key in out dict / cache.
    # And also move the key to the end
    # to show that it was recently used.
    def get(self, key: int) -> int:
        if key not in self.cache:
            print(f'{key} not in cache')
            return -1
        else:
            self.cache.move_to_end(key)
            return self.cache[key]
 
    # first, we add / update the key by conventional methods.
    # And also move the key to the end to show that it was recently used.
    # But here we will also check whether the length of our
    # ordered dictionary has exceeded our capacity,
    # If so we remove the first key (least recently used)
    def put(self, key: int, value: str) -> None:
        self.cache[key] = value
        self.cache.move_to_end(key)
        if len(self.cache) > self.capacity:
            self.cache.popitem(last = False)
 
 
# RUNNER
# initializing our cache with the capacity of 2
cache = LRUCache(3)
 
cache.put(1, 'New item-1')
print(cache.cache)
cache.put(2, 'New item-2')
print(cache.cache)
cache.get(1)
print(cache.cache)
cache.put(3, 'New item-3')
print(cache.cache)
cache.get(2)
print(cache.cache)
cache.put(4, 'New item-4')
print(cache.cache)
cache.get(5)
print(cache.cache)
cache.get(3)
print(cache.cache)
cache.get(4)
print(cache.cache)
        

OrderedDict([(1, 'New item-1')])
OrderedDict([(1, 'New item-1'), (2, 'New item-2')])
OrderedDict([(2, 'New item-2'), (1, 'New item-1')])
OrderedDict([(2, 'New item-2'), (1, 'New item-1'), (3, 'New item-3')])
OrderedDict([(1, 'New item-1'), (3, 'New item-3'), (2, 'New item-2')])
OrderedDict([(3, 'New item-3'), (2, 'New item-2'), (4, 'New item-4')])
5 not in cache
OrderedDict([(3, 'New item-3'), (2, 'New item-2'), (4, 'New item-4')])
OrderedDict([(2, 'New item-2'), (4, 'New item-4'), (3, 'New item-3')])
OrderedDict([(2, 'New item-2'), (3, 'New item-3'), (4, 'New item-4')])


In [6]:
import pandas as pd
import numpy as np
import random
import joblib
import torch
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn import model_selection
# from DataAugmentation 
from data_utils import loadDatafromFile,createTokenizedDf,CompDataset,createkfoldData,createDataloaders
from data_utils_unlabelled import loadUnlabelledDatafromFile,createTokenizedDfUnlabelled,CompDatasetUnlabelled
from DataAugmentation import DataAugmentation
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup
from importlib import reload
import config
from transformers import XLMRobertaForTokenClassification, XLMRobertaConfig ,BertModel, XLMRobertaTokenizer, XLMRobertaModel, BertForTokenClassification
from seqeval.metrics import accuracy_score, classification_report
from sklearn.metrics import matthews_corrcoef,classification_report,f1_score
import engine
# from model_new import EntityModel
import torch.nn as nn
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
from  loss_functions import ClassificationCost, ConsistencyCost, update_teacher_params
from utils_functions import set_seed, sigmoid_rampup, get_consistency_weight, EarlyStopping
from model_xlmroberta import EntityModel , GaussianNoise
from DataAugmentation import DataAugmentation

In [6]:
# import data_utils
# data_utils = reload(data_utils)
# from data_utils import loadDatafromFile,createTokenizedDf,CompDataset,createkfoldData,createDataloaders


In [7]:
# consistency_weight = 100.0 # max weights till you want a ramp-up, will be this value after rampup epoch is reached
# consistency_rampup = 1000 # till what epochs rampup is needed

In [116]:
# config = reload(config)

In [9]:
# lst = np.arange(0,1500,1)
# for item in lst :
#     if item==1000:
#         print(get_consistency_weight(item))

In [48]:
set_seed()    

In [49]:
import os
from torch.utils.tensorboard import SummaryWriter
from torch.utils.tensorboard.summary import hparams

class SummaryWriter(SummaryWriter):
    def add_hparams(self, hparam_dict, metric_dict):
        torch._C._log_api_usage_once("tensorboard.logging.add_hparams")
        if type(hparam_dict) is not dict or type(metric_dict) is not dict:
            raise TypeError('hparam_dict and metric_dict should be dictionary.')
        exp, ssi, sei = hparams(hparam_dict, metric_dict)

        logdir = self._get_file_writer().get_logdir()
        
        with SummaryWriter(log_dir=logdir) as w_hp:
            w_hp.file_writer.add_summary(exp)
            w_hp.file_writer.add_summary(ssi)
            w_hp.file_writer.add_summary(sei)
            for k, v in metric_dict.items():
                w_hp.add_scalar(k, v)

In [484]:
dataObj = loadDatafromFile(config.filePath_src,config.filePath_tar, config.filePath_srcTags,config.filePath_tarTags)
df= dataObj.createDf() 

df

,source,target,src_tokens,tar_tokens
0,José Ortega y Gasset visited Husserl at Freibu...,1934 besuchte José Ortega y Gasset Husserl in ...,OK OK OK OK OK OK OK OK OK OK OK,OK OK OK OK OK OK OK OK OK OK OK OK OK OK OK O...
1,"However , a disappointing ninth in China meant...",Eine enttäuschende Neunte in China bedeutete j...,OK OK OK BAD OK OK OK OK OK OK OK OK OK OK OK ...,OK BAD OK BAD OK BAD OK OK OK OK OK OK OK OK O...
2,"In his diary , Chase wrote that the release of...","In seinem Tagebuch , Chase schrieb , dass die ...",OK OK OK OK BAD BAD OK OK OK OK OK OK OK OK BA...,OK OK OK OK OK OK OK OK OK BAD OK BAD OK OK OK...
3,Heavy arquebuses mounted on wagons were called...,Schwere Arquebuses auf Waggons montiert wurden...,OK BAD BAD OK OK OK OK OK OK OK OK,OK OK OK BAD OK OK OK OK BAD BAD OK OK OK OK O...
4,Once North Pacific salmon die off after spawni...,Sobald der nordpazifische Lachs nach dem Laich...,OK OK OK OK BAD OK OK OK OK OK BAD OK BAD OK O...,OK OK OK BAD OK OK OK BAD OK OK OK OK OK OK OK...
...,...,...,...,...
6995,Some may also discourage or disallow unsanitar...,Einige können auch unhygienische Praktiken wie...,OK OK OK OK OK OK OK OK OK OK OK OK OK OK OK O...,OK OK OK OK OK OK OK OK OK OK OK OK OK OK OK O...
6996,"In the late 1860s , the crinolines disappeared...",In den späten 1860er Jahren verschwanden die K...,OK OK OK OK OK OK OK OK OK OK BAD BAD OK OK OK...,OK OK OK OK OK OK OK OK OK OK OK OK OK OK OK O...
6997,"Disco was criticized as mindless , consumerist...","Disco wurde als geistlos , konsumistisch , übe...",OK OK OK OK BAD OK BAD OK OK OK OK OK,OK OK OK OK OK OK OK BAD OK OK OK BAD OK OK OK...
6998,Planters would then fill large hogsheads with ...,Die Pflanzer würden dann große Heuschrecken mi...,OK OK OK BAD OK BAD OK OK BAD BAD OK OK OK OK OK,OK OK OK OK OK BAD OK OK OK OK OK BAD OK OK OK...


In [485]:
model_type='xlm'
dataaug_obj = DataAugmentation(df,swap_words=2,syn_words=4,del_words_prob=0.2,num_sentences=1)  
# swapDataset = dataaug_obj.random_swap()
# del_augDataset = dataaug_obj.random_deletion()
# del_augDataset = del_augDataset[7000:]
syn_dataset = dataaug_obj.random_deletion()
syn_dataset = syn_dataset[7000:8000]
syn_dataset =syn_dataset.reset_index(drop=True)
obj_tokenized_train = createTokenizedDf(syn_dataset,model_type)
df_new_syn = obj_tokenized_train.convertDf()
train_data_syn = CompDataset(df_new_syn,model_type)
syn_dataset

,source,target,src_tokens,tar_tokens
0,José y Gasset visited Husserl at in .,1934 besuchte José Ortega y Gasset Husserl in ...,OK OK OK OK OK OK OK OK,OK OK OK OK OK OK OK OK OK OK OK OK OK OK OK O...
1,However disappointing ninth in China meant he ...,Eine enttäuschende Neunte in China bedeutete j...,OK BAD OK OK OK OK OK OK OK OK OK OK,OK BAD OK BAD OK BAD OK OK OK OK OK OK OK OK O...
2,"In his diary , Chase wrote that the release of...","In seinem Tagebuch , Chase schrieb , dass die ...",OK OK OK OK BAD BAD OK OK OK OK OK OK OK BAD O...,OK OK OK OK OK OK OK OK OK BAD OK BAD OK OK OK...
3,Heavy arquebuses mounted on were arquebus à cr...,Schwere Arquebuses auf Waggons montiert wurden...,OK BAD BAD OK OK OK OK OK OK,OK OK OK BAD OK OK OK OK BAD BAD OK OK OK OK O...
4,"Once North Pacific salmon off after spawning ,...",Sobald der nordpazifische Lachs nach dem Laich...,OK OK OK OK OK OK OK OK OK BAD OK BAD OK OK OK...,OK OK OK BAD OK OK OK BAD OK OK OK OK OK OK OK...
...,...,...,...,...
995,"Marcus Valerius M. f . n Maximus Corvinus , co...","Marcus Valerius M. f. M. n. Maximus Corvinus ,...",OK OK OK OK OK OK OK OK OK OK OK OK OK,OK OK OK OK OK OK OK OK OK OK OK OK OK OK OK O...
996,"During the making , describes Vyjayanthimala ""...",Während der Entstehung des Führers beschreibt ...,OK OK OK OK OK OK OK OK OK OK OK OK,OK OK OK OK OK OK OK BAD OK BAD OK OK OK OK OK...
997,"defeating the attackers , he a designated rend...","Nachdem er die Angreifer besiegt hatte , ging ...",OK OK OK OK OK OK OK BAD OK OK OK OK,OK OK OK OK OK OK OK OK OK OK OK OK OK OK OK O...
998,"Finally , dear friend and defender , the Elect...",Schließlich starb 1714 sein lieber Freund und ...,OK OK OK BAD OK OK OK OK OK OK OK OK OK OK,OK OK OK OK OK OK OK BAD OK BAD OK BAD OK OK O...


In [486]:
syn_dataset.iloc[0].values

array(['José y Gasset visited Husserl at in .',
       '1934 besuchte José Ortega y Gasset Husserl in Freiburg .',
       'OK OK OK OK OK OK OK OK',
       'OK OK OK OK OK OK OK OK OK OK OK OK OK OK OK OK OK OK OK OK OK'],
      dtype=object)

In [511]:
def process_data_labelled(filePath_src,filePath_tar, filePath_srcTags,filePath_tarTags,
                 filePath_src_eval,filePath_tar_eval, filePath_srcTags_eval,filePath_tarTags_eval,train_data_needed,
                          unlabel_data_needed ,aug_data_needed=1000,total_train_data=2000, data_aug = False, model_type='xlm'):
    
    set_seed()
    dataObjTrain = loadDatafromFile(filePath_src,filePath_tar, filePath_srcTags,filePath_tarTags)
    df_train= dataObjTrain.createDf() # get dataframe from files    
 
    dataObjEval = loadDatafromFile(filePath_src_eval,filePath_tar_eval, filePath_srcTags_eval,filePath_tarTags_eval)
    df_test = dataObjEval.createDf()
    
    if data_aug:
    
        dataaug_obj = DataAugmentation(df_train,swap_words=2,syn_words=4,del_words_prob=0.2,num_sentences=1) 
        syn_dataset = dataaug_obj.random_deletion()
        syn_dataset = syn_dataset[7000:7000+aug_data_needed]
        syn_dataset =syn_dataset.reset_index(drop=True)
        print('aug data needed:',aug_data_needed)
        
    else:
         aug_data_needed=0
#     df_combined = df_train.append(df_eval, ignore_index=True)
#     df_combined.reset_index(drop=True)
    split_ratio_len = int(len(df_train) - config.test_split_ratio*len(df_train))
   
    df_train_splitted = df_train.iloc[:split_ratio_len,:]
    df_eval_splitted = df_train.iloc[split_ratio_len:,:]
    df_train_splitted = df_train_splitted[:total_train_data]#2000
    df_train_splitted_ = df_train_splitted[:train_data_needed] # amount of labeled data needed for training out of 2000 for eg. 250
    eval_data_needed_ssl = int(0.25 * (unlabel_data_needed)) # 25 % reserve for validation -->500 for 2k data train
    
    eval_data_needed_sl = int(0.25*(train_data_needed+aug_data_needed))
    df_eval_splitted_ssl = df_eval_splitted[:eval_data_needed_ssl] # fix this 500 for all experiments
    df_eval_splitted_sl = df_eval_splitted[:eval_data_needed_sl] 
    
#     df_train_splitted_ = df_train_splitted[:1000]
    df_unlabel_splitted_train = df_train_splitted[train_data_needed:] # get remaining data in train df to be used as unlabeled
    df_unlabel_splitted_train = df_unlabel_splitted_train.reset_index(drop=True)
#     df_eval_splitted = df_eval_splitted[:500]

#     df_unlabel_splitted_test = df_eval_splitted[eval_data_needed:]# get remaining data in eval df to use for unlabel
#     df_unlabel_splitted_test=df_unlabel_splitted_test.reset_index(drop=True)
    
#     df_unlabeled = df_unlabel_splitted_train.append(df_unlabel_splitted_test, ignore_index = True) # combine both unlabel df's
    if data_aug :
        df_combined = df_train_splitted_.append(syn_dataset,ignore_index = True)
    else :
        df_combined = df_train_splitted_.copy()
    
    obj_tokenized_train = createTokenizedDf(df_combined,model_type)
    obj_tokenized_eval_ssl = createTokenizedDf(df_eval_splitted_ssl,model_type)
    obj_tokenized_eval_sl = createTokenizedDf(df_eval_splitted_sl,model_type)
    obj_tokenized_test = createTokenizedDf(df_test,model_type)
    
    
    df_new_train = obj_tokenized_train.convertDf()
    df_new_eval_ssl = obj_tokenized_eval_ssl.convertDf()
    df_new_eval_sl = obj_tokenized_eval_sl.convertDf()
    df_new_test = obj_tokenized_test.convertDf()
    
    train_data = CompDataset(df_new_train,model_type)
    eval_data_ssl = CompDataset(df_new_eval_ssl,model_type)
    eval_data_sl = CompDataset(df_new_eval_sl,model_type)
    test_data = CompDataset(df_new_test,model_type)
    
    return df_combined, train_data, eval_data_ssl, eval_data_sl, test_data, df_unlabel_splitted_train   



In [537]:
total_train_data = 2000 
label_data_required = int(input('enter labeled data needed'))
aug_data_needed = int(input('augmented data needed'))
unlabel_data_required = total_train_data - label_data_required

enter labeled data needed 1250
augmented data needed 0


In [513]:
aug_data_needed

0

In [538]:
set_seed()
df, train_data , eval_data_ssl,eval_data_sl, test_data, df_unlabel = process_data_labelled(config.filePath_src,config.filePath_tar, config.filePath_srcTags,config.filePath_tarTags,
                                                config.filePath_src_eval,config.filePath_tar_eval, config.filePath_srcTags_eval,
                                                config.filePath_tarTags_eval,label_data_required,unlabel_data_required, aug_data_needed,total_train_data,data_aug=False)
df

,source,target,src_tokens,tar_tokens
0,José Ortega y Gasset visited Husserl at Freibu...,1934 besuchte José Ortega y Gasset Husserl in ...,OK OK OK OK OK OK OK OK OK OK OK,OK OK OK OK OK OK OK OK OK OK OK OK OK OK OK O...
1,"However , a disappointing ninth in China meant...",Eine enttäuschende Neunte in China bedeutete j...,OK OK OK BAD OK OK OK OK OK OK OK OK OK OK OK ...,OK BAD OK BAD OK BAD OK OK OK OK OK OK OK OK O...
2,"In his diary , Chase wrote that the release of...","In seinem Tagebuch , Chase schrieb , dass die ...",OK OK OK OK BAD BAD OK OK OK OK OK OK OK OK BA...,OK OK OK OK OK OK OK OK OK BAD OK BAD OK OK OK...
3,Heavy arquebuses mounted on wagons were called...,Schwere Arquebuses auf Waggons montiert wurden...,OK BAD BAD OK OK OK OK OK OK OK OK,OK OK OK BAD OK OK OK OK BAD BAD OK OK OK OK O...
4,Once North Pacific salmon die off after spawni...,Sobald der nordpazifische Lachs nach dem Laich...,OK OK OK OK BAD OK OK OK OK OK BAD OK BAD OK O...,OK OK OK BAD OK OK OK BAD OK OK OK OK OK OK OK...
...,...,...,...,...
1245,Music theorist and musicologist Joseph Yasser ...,Schon 1951 entdeckte der Musiktheoretiker und ...,OK OK OK OK OK OK OK OK OK OK OK OK OK OK OK O...,OK OK OK OK OK OK OK OK OK OK OK OK OK OK OK O...
1246,"The Rolleiflex Automat combined the two , wher...",Die Rolleiflex Automat kombinierte die beiden ...,OK OK OK OK OK OK OK OK BAD BAD BAD OK BAD OK ...,OK OK OK OK OK OK OK OK OK OK OK OK OK OK OK O...
1247,"In 2014 Nolan Burch , 18 , of 20 Kappa Sigma d...","Im Jahr 2014 starb Nolan Burch , 18 , von 20 K...",OK OK BAD BAD OK BAD OK OK OK OK OK OK OK OK O...,OK OK OK OK OK OK OK OK OK BAD OK BAD OK BAD O...
1248,"Cod , haddock , whiting , saithe , plaice , so...","Kabeljau , Schellfisch , Wittling , Seelachs ,...",OK OK OK OK OK OK OK OK OK OK OK OK OK OK OK O...,OK OK OK OK OK OK OK OK OK OK OK OK OK OK OK O...


In [491]:
df.iloc[0].values

array(['José Ortega y Gasset visited Husserl at Freiburg in 1934 .',
       '1934 besuchte José Ortega y Gasset Husserl in Freiburg .',
       'OK OK OK OK OK OK OK OK OK OK OK',
       'OK OK OK OK OK OK OK OK OK OK OK OK OK OK OK OK OK OK OK OK OK'],
      dtype=object)

In [492]:
# df.iloc[1000].values

In [539]:
len_labelled_data = len(train_data)
len_eval_data_ssl = len(eval_data_ssl)
print('labeled data',len_labelled_data)
print('ssl eval data',len_eval_data_ssl)
print('sl eval data',len(eval_data_sl))
print('unlabel data',len(df_unlabel))
print('test data',len(test_data))

labeled data 1250
ssl eval data 187
sl eval data 312
unlabel data 750
test data 1000


In [516]:
print(next(iter(train_data))[0]) # 0,17151,12456,..
print(next(iter(eval_data_ssl))[0])#121220,..

tensor([     0,  17151,  12426,   2765,    113,    527, 110896,  36997,     71,
         20387,   2189,    141,     99, 183124,     23,  58020,      6,      5,
             2,      2,    656,  58020,    656, 138438,     13,    656,  17151,
           656,  12426,   2765,    656,    113,    656,    527, 110896,    656,
         20387,   2189,    141,    656,     23,    656, 183124,    656,      6,
             5,    656,      2,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1, 

In [540]:
loader_obj = createDataloaders(train_data,config.TRAIN_BATCH_SIZE)
train_dataloader = loader_obj.createDataloaders()
print(len(train_dataloader))
loader_obj = createDataloaders(eval_data_sl,config.TRAIN_BATCH_SIZE)
val_dataloader = loader_obj.createDataloaders()
print(len(val_dataloader))
loader_obj = createDataloaders(test_data,config.TRAIN_BATCH_SIZE)
test_dataloader = loader_obj.createDataloaders()
len(test_dataloader)


157
39


125

In [272]:
experiment_name_supervised =f'augsupervised_del_{label_data_required}_labeled_data_{aug_data_needed}_aug_data'
model_name_supervised = f'augsupervised_del_with_{label_data_required}_labeled_data_{aug_data_needed}_aug_data'
#augsupervised_1000_labeled_data_1000_aug_data
#augsupervised_with_1000_labeled_data_1000_aug_data
model_name_supervised

'augsupervised_del_with_1000_labeled_data_1000_aug_data'

In [273]:
writer_supervised = SummaryWriter(f"runs/AugSupervised_and_PseudoLabelingTraining/augsupervised_del_{label_data_required}_labeled_data_{aug_data_needed}_aug_data")

In [275]:
model = EntityModel(with_noise_layer = False, dropout_layer=True)
model.cuda()

In [276]:
param_optimizer = list(model.named_parameters())
no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
optimizer_parameters = [
    {
        "params": [
            p for n, p in param_optimizer if not any(nd in n for nd in no_decay)
        ],
        "weight_decay": 0.001,
    },
    {
        "params": [
            p for n, p in param_optimizer if any(nd in n for nd in no_decay)
        ],
        "weight_decay": 0.0,
    },
]
num_train_steps = int(len(train_data) / config.TRAIN_BATCH_SIZE * config.EPOCHS) #10 is the batchsize
optimizer = AdamW(optimizer_parameters, lr=2e-5) # used 3e-5
scheduler = get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps=0, num_training_steps=num_train_steps
)
early_stopping = EarlyStopping(patience=8)

/opt/conda/lib/python3.6/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
set_seed()
best_accuracy = -1
train_loss_lst = []
val_loss_lst = []
with open(f'../Logs/AugSupervised_logs/{experiment_name_supervised}', 'w') as f:
    f.write(f'{experiment_name_supervised}'+'\n')
    for epoch in range(config.EPOCHS):
            print(f'Epoch {epoch+1} of {config.EPOCHS}')
            train_metrics = engine.train_fn(train_dataloader, model, optimizer, scheduler)
            print(classification_report(train_metrics[21],train_metrics[22]))
            test_metrics = engine.eval_fn(val_dataloader, model)
            print(classification_report(test_metrics[21],test_metrics[22]))
#             print('For gaps : ',classification_report(test_metrics[27],test_metrics[28]) )
#             print('For tar :',classification_report(test_metrics[25],test_metrics[26]) )
#             print('For source : ',classification_report(test_metrics[23],test_metrics[24]) )
            print(f"Train Loss = {train_metrics[0]} Valid Loss = {test_metrics[0]}")
            writer_supervised.add_scalar('Train_loss/Supervised',train_metrics[0], epoch)
            writer_supervised.add_scalar('Validation_loss/Supervised',test_metrics[0],epoch)
            writer_supervised.add_scalar('mcc_score/Supervised_validation',test_metrics[4],epoch)
            writer_supervised.add_scalar('mcc_score/Supervised_train',train_metrics[4],epoch)
            train_loss_lst.append(train_metrics[:-14])
            val_loss_lst.append(test_metrics[:-14])
            f.write(f"Train_loss {epoch+1} : {str(train_loss_lst)}" + '\n')
            f.write(f"val_loss {epoch+1} : {str(val_loss_lst)}" + '\n')
            
            if early_stopping.step(float("{:.2f}".format(test_metrics[4]))): #mcc score for early stopping "{:.2f}".format(a_float)
                  break  # early stop criterion is met, we can stop now
            if float("{:.2f}".format(test_metrics[4])) >  best_accuracy:
                torch.save(model.state_dict(), f'../models/augmented_data/{model_name_supervised}.bin')
                best_accuracy = float("{:.2f}".format(test_metrics[4]))
                print('Model saved for this epoch!')
  

  0%|          | 0/250 [00:00<?, ?it/s]

Epoch 1 of 25


/root/Thesis/src/code/code_files/engine.py:409: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  np_arr_labels = np.array(tar_labels)
  0%|          | 0/63 [00:00<?, ?it/s]

              precision    recall  f1-score   support

           0       0.22      0.78      0.35     17706
           1       0.95      0.60      0.73    119610

    accuracy                           0.62    137316
   macro avg       0.59      0.69      0.54    137316
weighted avg       0.86      0.62      0.68    137316



/root/Thesis/src/code/code_files/engine.py:409: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  np_arr_labels = np.array(tar_labels)
100%|██████████| 63/63 [00:03<00:00, 16.54it/s]


...

Average F1 Validation score for whole sentence class 0 : 0.34191222180778313
Average F1 Validation score for whole sentence class 1 : 0.9138915550928111
Average Accuracy Validation score whole sentence  : 0.8477097479571872
Average mcc Validation score whole sentence : 0.2583559320944871
Average F1 Validation score for source sentence class 0 : 0.3277511961722488
Average F1 Validation score for source sentence class 1 : 0.8994723191127806
Average Accuracy Validation score source sentence  : 0.8250991986306698
Average mcc Validation score source sentence : 0.22985839005477063
Average F1 Validation score for target sentence class 0 : 0.3752645191629438
Average F1 Validation score for target sentence class 1 : 0.8830803080308031
Average Accuracy Validation score target sentence  : 0.8030246867818223
Average mcc Validation score target sentence : 0.2619454295005045
Average F1 Validation score for gaps in target sentence class 0 : 0.13004484304932734
Average F1 Validation score for gaps in 

/root/Thesis/src/code/code_files/engine.py:409: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  np_arr_labels = np.array(tar_labels)
  0%|          | 0/63 [00:00<?, ?it/s]

              precision    recall  f1-score   support

           0       0.78      0.97      0.86     17706
           1       0.99      0.96      0.98    119610

    accuracy                           0.96    137316
   macro avg       0.89      0.96      0.92    137316
weighted avg       0.97      0.96      0.96    137316



/root/Thesis/src/code/code_files/engine.py:409: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  np_arr_labels = np.array(tar_labels)
100%|██████████| 63/63 [00:03<00:00, 15.91it/s]


Average F1 Validation score for whole sentence class 0 : 0.34809272660370416
Average F1 Validation score for whole sentence class 1 : 0.9133786404602423
Average Accuracy Validation score whole sentence  : 0.847076763724249
Average mcc Validation score whole sentence : 0.26334581109778266
Average F1 Validation score for source sentence class 0 : 0.32965064198268135
Average F1 Validation score for source sentence class 1 : 0.8995840229011048
Average Accuracy Validation score source sentence  : 0.8253326071734225
Average mcc Validation score source sentence : 0.23179505645661336
Average F1 Validation score for target sentence class 0 : 0.38518863530507685
Average F1 Validation score for target sentence class 1 : 0.8836184094515958
Average Accuracy Validation score target sentence  : 0.8042849729409148
Average mcc Validation score target sentence : 0.27178945108148667
Average F1 Validation score for gaps in target sentence class 0 : 0.1568627450980392
Average F1 Validation score for gaps i

In [50]:
# # Read the labelled data
# def process_data_labelled(filePath_src,filePath_tar, filePath_srcTags,filePath_tarTags,
#                  filePath_src_eval,filePath_tar_eval, filePath_srcTags_eval,filePath_tarTags_eval,
#                  model_type='xlm'):
    
    
#     dataObjTrain = loadDatafromFile(filePath_src,filePath_tar, filePath_srcTags,filePath_tarTags)
#     df_train= dataObjTrain.createDf() # get dataframe from files    
#     df_train = df_train.sample(frac=1).reset_index(drop=True)
#     df_train=df_train.iloc[:500,:]
#     dataObjEval = loadDatafromFile(filePath_src_eval,filePath_tar_eval, filePath_srcTags_eval,filePath_tarTags_eval)
#     df_eval = dataObjEval.createDf()
#     obj_tokenized_train = createTokenizedDf(df_train,model_type)
#     obj_tokenized_test = createTokenizedDf(df_eval,model_type)
#     df_new_train = obj_tokenized_train.convertDf()
#     df_new_eval = obj_tokenized_test.convertDf()
#     train_data = CompDataset(df_new_train,model_type)
#     test_data = CompDataset(df_new_eval,model_type)
#     return train_data,test_data


In [278]:
model.load_state_dict(torch.load(f'../models/augmented_data/{model_name_supervised}.bin'))
model.eval()
test_metrics = engine.eval_fn(test_dataloader, model)
mcc_score_supervised = test_metrics[4]  

  0%|          | 0/125 [00:00<?, ?it/s]/root/Thesis/src/code/code_files/engine.py:409: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  np_arr_labels = np.array(tar_labels)
100%|██████████| 125/125 [00:07<00:00, 16.13it/s]


Average F1 Validation score for whole sentence class 0 : 0.39738322482776056
Average F1 Validation score for whole sentence class 1 : 0.8247394253570838
Average Accuracy Validation score whole sentence  : 0.7284534320706352
Average mcc Validation score whole sentence : 0.3013759352012734
Average F1 Validation score for source sentence class 0 : 0.38081458438833826
Average F1 Validation score for source sentence class 1 : 0.7742701163066699
Average Accuracy Validation score source sentence  : 0.6691534596057209
Average mcc Validation score source sentence : 0.2571879817264198
Average F1 Validation score for target sentence class 0 : 0.4210647614657755
Average F1 Validation score for target sentence class 1 : 0.7291498193972721
Average Accuracy Validation score target sentence  : 0.6309545671576009
Average mcc Validation score target sentence : 0.26634415636998077
Average F1 Validation score for gaps in target sentence class 0 : 0.004319654427645789
Average F1 Validation score for gaps i

In [279]:
writer_supervised.add_hparams(
                    { "lr": float(config.lr),  
                     "labelled_data": float(len(train_data)), 
                     "validation_data": float(len(eval_data_sl)),
                     "alpha":0.0, 
                     "teacher_noise_gaussian_std":0.0,
                     "student_noise_gaussian_std":0.0,
                     "unlabelled_data":0.0, 
                    },
                    {
                    "mcc_score_test" : float(mcc_score_supervised),
                    "mcc_score_teacher": float(0) ,
                    "mcc_score_student" : float(0),
                    },
                )

In [541]:
def process_data_unlabelled(filePath_src,filePath_tar,df_unlabel, unlabel_data_needed, model_type):
    
    set_seed()
    dataObj = loadUnlabelledDatafromFile(filePath_src,filePath_tar)
    df= dataObj.createDfUnlabelled() # get dataframe from files
    df['len_src'] = df['source'].str.split().map(lambda x:len(x))
    df['len_tar'] = df['target'].str.split().map(lambda x:len(x))
    
    df = df[df.len_tar != 36]
    df = df[df.len_tar != 2]
    df = df[df.len_src != 1]
    
    df=df.iloc[:,:-2]
#     df=df.iloc[:1000,:]
#     df = df[:2000]
    if unlabel_data_needed > len(df_unlabel):
        
        extra_data_needed = unlabel_data_needed - len(df_unlabel) # TAKE IT FROM THE BACKTRANSLATED DATASET
        df = df[:extra_data_needed]
        df_unlabel = df_unlabel.append(df,ignore_index=True)
    
    else:
        
        df_unlabel = df_unlabel[:unlabel_data_needed]
        
    obj_tokenized = createTokenizedDfUnlabelled(df_unlabel,model_type) # change to df
    df_new = obj_tokenized.convertDf()
#     enc_label = preprocessing.LabelEncoder()
#     df_new['labels']= enc_label.fit_transform(df_new['labels'])
    train_data = CompDatasetUnlabelled(df_new,model_type)
    return df_new,train_data

In [542]:
df_unlabelled_train, dataset_train_unlabelled = process_data_unlabelled(config.filePath_src_backtranslated,config.filePath_tar_backtranslated,df_unlabel,
                                                                       unlabel_data_required, model_type = 'xlm')
len_unlabelled_data = len(dataset_train_unlabelled)
print(len_unlabelled_data)
temp_val = next(iter(dataset_train_unlabelled)) # 31384,..
temp_val

750


(tensor([     0,  10660,  90254,     33,    297,   2363,  61475,  84616,    678,
            158,   1916,  35066,   1295, 105355,   1174,      6,      4,   1172,
            408,   3390,      6,      4,   5631,     73,    927,      6,      4,
            136,  26842,    219,      6,      5,      2,      2,    656,    727,
            656,      6,  43999,    510,    656,   9047,    656,    142,    420,
          20954,  12512,    656, 227144,    656,    491,    656,  42990,   6475,
            510,    656,   1312,    656, 105355,   1174,    656,      6,      4,
            656,   1172,    408,   3390,    656,      6,      4,    656,   3970,
             73,    927,    656,    165,    656,  26842,    219,    656,      6,
              5,    656,      2,      1,      1,      1,      1,      1,      1,
              1,      1,      1,      1,      1,      1,      1,      1,      1,
              1,      1,      1,      1,      1,      1,      1,      1,      1,
              1,      1,    

In [543]:
# def process_data_unlabelled(filePath_src,filePath_tar,len_train_data,df_unlabelled_train,model_type):
    
#     dataObj = loadUnlabelledDatafromFile(filePath_src,filePath_tar)
#     df= dataObj.createDfUnlabelled() # get dataframe from files
#     df['len_src'] = df['source'].str.split().map(lambda x:len(x))
#     df['len_tar'] = df['target'].str.split().map(lambda x:len(x))
    
#     df = df[df.len_tar != 36]
#     df = df[df.len_tar != 2]
#     df = df[df.len_src != 1]
    
#     df = df.iloc[:,:-2]
#     df = df.reset_index(drop=True)
#     df_unlabelled_train = df_unlabelled_train.iloc[:,:-2]
#     df_new = df.append(df_unlabelled_train, ignore_index = True)
#     len_unlabelled_needed = config.fix_train_size - len_train_data
#     df_new=df_new[:len_unlabelled_needed]
# #     print(df_new)
    
#     obj_tokenized = createTokenizedDfUnlabelled(df_new,model_type)
#     df_unlabelled= obj_tokenized.convertDf()
# #     enc_label = preprocessing.LabelEncoder()
# #     df_new['labels']= enc_label.fit_transform(df_new['labels'])
#     train_data = CompDatasetUnlabelled(df_unlabelled,model_type)
#     return df_new,train_data


In [544]:
unlabel_data_required

750

In [545]:
#combining 2 tuples --> labelled and unlabelled data 
combined_Data =  dataset_train_unlabelled
len(combined_Data)

750

In [546]:
# dataloaders
loader_obj = createDataloaders(combined_Data,config.TRAIN_BATCH_SIZE)
train_dataloader = loader_obj.createDataloaders()
len(train_dataloader)

94

In [547]:
loader_obj = createDataloaders(eval_data_ssl,config.VALID_BATCH_SIZE)
val_dataloader = loader_obj.createDataloaders()
len(val_dataloader)

24

In [549]:
# config = reload(config)

In [550]:
# noise = GaussianNoise(0.7)
# print(noise(a))
# print(torch.randn(a.size()) * 0.7 )
# print(torch.autograd.Variable(torch.randn(a.size()).cuda() * 0.7))

In [551]:
# model = EntityModel(std_gaussian=config.gaussian_noise_std_student, with_noise_layer=True )
# model.cuda()# print(noise ,embeddings)

In [552]:
# outputs, outputs1 = model(batch[0].cuda(), attention_mask = batch[1].cuda())

In [553]:
# model2 = EntityModel(std_gaussian=config.gaussian_noise_std_teacher)
# model2.cuda()
# outputs, outputs1 = model2(batch[0].cuda(), attention_mask = batch[1].cuda())
# print(outputs,'\n')
# print(outputs1)

In [554]:
def ClassificationCost(output,hard_labels_pseudo,mask,label_mask):
    
    
    active_loss = label_mask.view(-1) & mask.view(-1)#loss calculation for non padded tokens only (mask =1)
    active_loss = active_loss.bool()
    
    active_logits = output.view(-1,2)
    
    active_labels = torch.where( # just append -100 for the padded tokens so its ignored when computing loss , no need now
        active_loss,             # since its handled in preprocessing only
        hard_labels_pseudo.view(-1),
        torch.tensor(-100).type_as(hard_labels_pseudo)    
    )
    try:
        class_0_weights =1/len(torch.where(active_labels==0)[0]) # trying to weight the labels as its unbalanced mostly
    
    except ZeroDivisionError:
        class_0_weights = float(1)

    try:
        class_1_weights =1/len(torch.where(active_labels==1)[0])
    
    except:
        class_1_weights = float(1)
    
    weights_tensor = torch.tensor([class_0_weights,class_1_weights]).cuda()
    lfn = nn.CrossEntropyLoss(weight = weights_tensor)
    loss = lfn(active_logits,active_labels)
    
    return loss

In [555]:
def ClassificationCost_eval(output,target,mask):
    
    
    active_loss = mask.view(-1) == 1 #loss calculation for non padded tokens only (mask =1)
    active_logits = output.view(-1,2)
    
    active_labels = torch.where( # just append -100 for the padded tokens so its ignored when computing loss , no need now
        active_loss,             # since its handled in preprocessing only
        target.view(-1),
        torch.tensor(-100).type_as(target)    
    )
    try:
        class_0_weights =1/len(torch.where(active_labels==0)[0]) # trying to weight the labels as its unbalanced mostly
    
    except ZeroDivisionError:
        class_0_weights = 1e-8

    try:
        class_1_weights =1/len(torch.where(active_labels==1)[0])
    
    except:
        class_1_weights = 1e-8
    
    weights_tensor = torch.tensor([class_0_weights,class_1_weights]).cuda()
    lfn = nn.CrossEntropyLoss(weight = weights_tensor)
    loss = lfn(active_logits,active_labels)
    
    return loss

In [556]:
# a = torch.tensor([[0.2,0.5],[0.9,0.4],[0.9,0.2],[0.2,0.9],[0.7,0.4]])
# b = torch.tensor([-100,-100,-100,-100,-100])
# c = torch.tensor([1,1,1,1,1])
# ClassificationCost(a,b,c).item()

In [557]:
def ConsistencyCost(student_output, teacher_output,mask,scale=10):
    
    assert len(student_output) == len(teacher_output)
    
    loss = nn.MSELoss()
    active_outputs = mask.view(-1) == 1 #loss calculation for non padded tokens only (mask =1)
    flattened_outputs_student = student_output.view(-1,2)
    flatenned_outputs_teacher = teacher_output.view(-1,2)

    active_outputs_teacher=flatenned_outputs_teacher[torch.where(mask.view(-1) == 1)]
    active_outputs_student=flattened_outputs_student[torch.where(mask.view(-1) == 1)]
    
    return scale * loss(active_outputs_student.view(-1),active_outputs_teacher.view(-1))
    

In [558]:
# def ConsistencyCost(student_output, teacher_output,mask):
    
#     assert len(student_output) == len(teacher_output)
    
#     loss = nn.MSELoss()
#     active_outputs = mask.view(-1) == 1 #loss calculation for non padded tokens only (mask =1)
#     flattened_outputs_student = student_output.view(-1,2)
#     flatenned_outputs_teacher = teacher_output.view(-1,2)
    
#     # [0.2,0.8] , [0.3,0.7] , [0.9,0.1] -> s
#     # [0.7,0.3] , [0.9,0.1] , [0.8,0.2] -> T 
    
    
    
#     pseudo_label = torch.softmax(logits_u_w.detach()/args.T, dim=-1)
#     max_probs, targets_u = torch.max(pseudo_label, dim=-1)
#     mask = max_probs.ge(config.threshold).float()

#     Lu = (F.cross_entropy(logits_u_s, targets_u,
#                                   reduction='none') * mask)
    
#     active_outputs_teacher=flatenned_outputs_teacher[torch.where(active_outputs == 1)]
#     active_outputs_student=flattened_outputs_student[torch.where(active_outputs == 1)]
    
#     return torch.sqrt(loss(active_outputs_student,active_outputs_teacher))
    

In [559]:
# Update teacher to be exponential moving average of student params.
def update_teacher_params(student_model, teacher_model, alpha, global_step):
    # Use the true average until the exponential average is more correct
    
    teacher_params = teacher_model.parameters()
    student_params = student_model.parameters()
    
    assert sum(p.numel() for p in student_model.parameters()) == sum(p.numel() for p in teacher_model.parameters())
    
#     alpha = min(1 - 1 / (global_step + 1), alpha)
    
    for teacher_param, student_param in zip(teacher_params, student_params):
            teacher_param.data.mul_(config.alpha).add_(1 - config.alpha, student_param.data)
    

In [560]:
def refine_labels(teacher_softmax):
    
#             pseudo_label = torch.softmax(logits_u_w.detach()/args.T, dim=-1)
#             print(torch.softmax(logits, dim=-1))
            max_probs, targets_hard = torch.max(teacher_softmax, dim=-1)
            mask = max_probs.ge(config.threshold).int()
            
    
            return mask, targets_hard

In [561]:
# import torch
# import torch.nn as nn 
# import config
lfn = nn.CrossEntropyLoss()
logits_t = torch.tensor([[[0.435345,-0.3454],[-0.4254,0.4354],[-0.324,0.23434]],[[0.456546,0.12334],[0.657,-0.9564],[0.234,-0.23443]]]).cuda()
logits_st = torch.tensor([[[0.635345,-0.1454],[0.4254,0.134],[-0.324,0.23434]],[[-0.456546,0.65334],[0.657,0.1564],[-0.324,0.23434]]]).cuda()
att_mask=torch.tensor([[1,1,0],[1,1,0]]).cuda()
softmax_teacher = torch.softmax(logits_t, dim=-1)
print(softmax_teacher)
label_mask,hard_labels_pseudo = refine_labels(softmax_teacher)
print(label_mask)
print(hard_labels_pseudo)
active_loss = label_mask.view(-1) & att_mask.view(-1)
print(active_loss.bool())
active_loss = active_loss.bool()
active_labels = torch.where( # just append -100 for the padded tokens so its ignored when computing loss , no need now
        active_loss,             # since its handled in preprocessing only
        hard_labels_pseudo.view(-1),
        torch.tensor(-100).type_as(hard_labels_pseudo)    
    )
print(active_labels)
active_logits = logits_st.view(-1,2)
# active_logits = logits_st.view(-1, 2)[active_loss]
# active_labels = hard_labels_pseudo.view(-1, 2)[active_loss]
print(active_logits)
# print(active_labels)
loss = lfn(active_logits,active_labels)
print(loss.item())
# max_probs, targets_u = torch.max(pseudo_label, dim=-1)
# mask = max_probs.ge(0.7).float()
# print(pseudo_label)
# targets_u

tensor([[[0.6858, 0.3142],
         [0.2972, 0.7028],
         [0.3639, 0.6361]],

        [[0.5825, 0.4175],
         [0.8339, 0.1661],
         [0.6150, 0.3850]]], device='cuda:0')
tensor([[0, 0, 0],
        [0, 1, 0]], device='cuda:0', dtype=torch.int32)
tensor([[0, 1, 1],
        [0, 0, 0]], device='cuda:0')
tensor([False, False, False, False,  True, False], device='cuda:0')
tensor([-100, -100, -100, -100,    0, -100], device='cuda:0')
tensor([[ 0.6353, -0.1454],
        [ 0.4254,  0.1340],
        [-0.3240,  0.2343],
        [-0.4565,  0.6533],
        [ 0.6570,  0.1564],
        [-0.3240,  0.2343]], device='cuda:0')
0.4738504886627197


In [562]:
def train(train_dataloader, optimizer,scheduler, student_model, teacher_model , global_step,with_const_cost, epoch):
    
    student_model.train() # put student model in training mode
    teacher_model.train()
    total_train_loss = 0
    consistency_cst = 0
    classification_cst_student=0
    classification_cst_teacher=0
    classification_cst_lst = []
    consistency_cst_lst = []
    overall_cst_lst = []
#     global_step = 0
    
    for step, batch in enumerate(tqdm(train_dataloader, total = len(train_dataloader),position = 0 , leave = True)):

            
        b_input_ids = batch[0].cuda()
        b_input_mask = batch[1].cuda()
        
     
        
        output_student_softmax, output_student_logit = student_model( b_input_ids, attention_mask = b_input_mask )
        
        # make teacher's weights ema of student's weights.
        update_teacher_params(student_model , teacher_model , config.alpha,global_step)
        
        with torch.no_grad():
            
                output_teacher_softmax, output_teacher_logit = teacher_model( b_input_ids, attention_mask = b_input_mask )
          
        # get hard labels and mask for confident labels 
        label_masks,hard_labels_teacher_pseudo = refine_labels(output_teacher_softmax)
        # classification cose between student's logits and teacher's hard pseudo labels (only confident ones)
        
        classification_cost_student =  ClassificationCost(output_student_logit,hard_labels_teacher_pseudo,b_input_mask,label_masks)
        classification_cst_student+=classification_cost_student.item()
        
#         classification_cost_teacher =  ClassificationCost(output_teacher_logit,b_labels,b_input_mask)
#         classification_cst_teacher+=classification_cost_teacher.item()
        
        writer.add_scalar('TrainingLoss/Classification', classification_cost_student.item(), global_step)
        
        consistency_cost = ConsistencyCost(output_student_softmax,output_teacher_softmax.detach(),b_input_mask,scale=10)
        consistency_cst+=consistency_cost.item()
        
        writer.add_scalar('TrainingLoss/Consistency', consistency_cost.item(), global_step)
        
#         overall_cost = (config.ratio * classification_cost) + ((1 - config.ratio) * consistency_cost)
        
        if with_const_cost:
            overall_cost =  classification_cost_student + get_consistency_weight(global_step) * consistency_cost
        else:
            overall_cost =  classification_cost_student
        
        writer.add_scalar('TrainingLoss/Overall', overall_cost.item(), global_step)
        
        total_train_loss+=overall_cost.item()

        overall_cost.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
        
#         update_teacher_params(student_model , teacher_model , config.alpha,global_step)

        
        if step % 50 == 0 and step!=0:
            print ('train loss : ', total_train_loss/step)
            print('Consistency Cost :', consistency_cst/step)
            print('Classification Cost :', classification_cst_student/step)
            print('Consistency Weight is:', get_consistency_weight(epoch))
    
        
        global_step+=1
        
    return (consistency_cst/len(train_dataloader), classification_cst_student/len(train_dataloader), total_train_loss/len(train_dataloader),global_step)

In [563]:
# def soft_frequency(logits, power=2, probs=False):
#     """
#     Unsupervised Deep Embedding for Clustering Analysis
#     https://arxiv.org/abs/1511.06335
#     """
#     if not probs:
#         softmax = torch.nn.Softmax(dim=1)
#         y = softmax(logits.view(-1, logits.shape[-1])).view(logits.shape)
#     else:
#         y = logits
#     f = torch.sum(y, dim=(0, 1))
#     t = y**power / f
#     p = t/torch.sum(t, dim=2, keepdim=True)

#     return p

In [564]:
# logits = torch.tensor([[[0.3,0.7] , [0.2,0.8] , [0.5569,0.5]]])
# f = torch.sum(logits, dim=(0,1))
# print(f)
# new_logits = soft_frequency(logits,probs = True)
# # t = logits**2 / f
# # print(t.shape)
# # p = t/torch.sum(t, dim=2, keepdim=True)
# # p

In [565]:
# softmax = torch.nn.Softmax(dim=1)
# y = softmax(new_logits.view(-1, new_logits.shape[-1])).view(new_logits.shape)
# y

In [566]:
# _confidence = y.max(dim=-1)[0]
# _confidence[0]
# # torch.where(_confidence[0]>0.5)

In [567]:
def test( model, test_loader):
    
    model.eval()
    test_loss = 0
    correct = 0
    lst_active_preds = []
    lst_active_labels = []
    with torch.no_grad():
        
        for batch in tqdm(test_loader,total = len(test_loader),position = 0 , leave = True):
            
            b_input_ids_eval = batch[0].cuda()
            b_input_mask_eval = batch[1].cuda()
            b_labels_eval = batch[2].cuda()
            
            _, output= model(b_input_ids_eval,attention_mask = b_input_mask_eval)
            
            classification_cost =  ClassificationCost_eval(output,b_labels_eval,b_input_mask_eval)
            test_loss+=classification_cost
            
            labels = b_labels_eval.view(-1) 
            active_logits = output.view(-1, 2)
            flattened_predictions = torch.argmax(active_logits, axis=1)
            active_accuracy = labels.view(-1) != -100
            labels_tmp = torch.masked_select(labels, active_accuracy) 
            pred_tmp = torch.masked_select(flattened_predictions, active_accuracy) 
            lst_active_labels.extend(labels_tmp.tolist())
            lst_active_preds.extend(pred_tmp.tolist())
            
    avg_f1_score_0=f1_score(lst_active_labels,lst_active_preds,average='binary',pos_label = 0)
    avg_f1_score_1=f1_score(lst_active_labels,lst_active_preds,average='binary',pos_label = 1)
    avg_accuracy_score=accuracy_score(lst_active_labels,lst_active_preds)
    avg_mcc_score = matthews_corrcoef(lst_active_labels,lst_active_preds)
    
    test_loss /= len(test_loader)
    
    print('Overall validation loss:', test_loss)
    print('Average F1 Validation score for whole sentence class 0 :' ,avg_f1_score_0)
    print('Average F1 Validation score for whole sentence class 1 :' ,avg_f1_score_1)
    print('Average Accuracy Validation score whole sentence  :' ,avg_accuracy_score)
    print('Average mcc Validation score whole sentence :' ,avg_mcc_score)
    print('Classification Report :'+'\n', classification_report(lst_active_labels,lst_active_preds))

    return (test_loss, avg_f1_score_0 , avg_f1_score_1, avg_accuracy_score, avg_mcc_score)

In [568]:
def train_MeanTeacher(train_dataloader, val_dataloader,len_labelled_data ,len_unlabelled_data,exp_name, model_name, early_stopping,writer,with_const_cost=True ,dropout_layer = True, noise_layer=True):
    
    exp_name = exp_name
    model_name = model_name
    
    global_step = 0
    best_mcc_teacher = -1
    best_mcc_student = -1
    # train losses
    epochs_consistency_lst =[]
    epochs_classification_lst = []
    epochs_overall_lst = []
    
    #test metrices for student
    epochs_f1_score_0_lst_student = []
    epochs_f1_score_1_lst_student = []
    epochs_accuracy_lst_student = []
    epochs_mcc_lst_student = []
    epochs_cost_lst_student=[]
    
    #test metrices for teacher
    epochs_f1_score_0_lst_teacher= []
    epochs_f1_score_1_lst_teacher = []
    epochs_accuracy_lst_teacher = []
    epochs_mcc_lst_teacher = []
    epochs_cost_lst_teacher=[]
    
    set_seed()
    #initialaize models with pretrained model 
    if with_const_cost:
        noise_layer = True
    else:
        noise_layer = False
        
    student = EntityModel(std_gaussian=config.gaussian_noise_std_student, with_noise_layer=noise_layer,dropout_layer=dropout_layer) 
    teacher = EntityModel(std_gaussian=config.gaussian_noise_std_teacher, with_noise_layer=noise_layer)
#     Thesis/src/code/models/training_data/supervised_with_250_labeled_data.bin
    student.load_state_dict(torch.load(f'../models/training_data/supervised_with_{len_labelled_data}_labeled_data.bin'))
    teacher.load_state_dict(torch.load(f'../models/training_data/supervised_with_{len_labelled_data}_labeled_data.bin'))
    student.cuda() # take model to gpu
    teacher.cuda()
#     teacher.eval()
    param_optimizer_student = list(student.named_parameters())
    param_teacher = list(teacher.named_parameters())
    
    no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
    optimizer_parameters = [
    {
        "params": [
            p for n, p in param_optimizer_student if not any(nd in n for nd in no_decay)
        ],
        "weight_decay": 0.001,
    },
    {
        "params": [
            p for n, p in param_optimizer_student if any(nd in n for nd in no_decay)
        ],
        "weight_decay": 0.0,
    },
]
    optimizer = AdamW(optimizer_parameters, lr=config.lr)
    num_train_steps = int(len(train_dataloader) * config.EPOCHS)
#     scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
#         optimizer, T_max=num_train_steps)
    scheduler = get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps=0, num_training_steps=num_train_steps)
    set_seed() # setting seed again before training
    
    with open(f'../Logs/{exp_name}.txt', 'w') as f:
        f.write(f'{exp_name}'+'\n')
        for epoch in range(0,config.EPOCHS):
            print(f'Current epoch is {epoch+1} of {config.EPOCHS}')

            consistency_cost , classification_cost , overall_cost, global_step = train(train_dataloader, optimizer,scheduler,student,teacher,global_step,with_const_cost,epoch)

            writer.add_scalar('TrainingLoss/Consistency_epoch :', consistency_cost,epoch )
            writer.add_scalar('TrainingLoss/Classification_epoch', classification_cost,epoch)
            writer.add_scalar('TrainingLoss/Overall_epoch',overall_cost,epoch)

            epochs_consistency_lst.append(consistency_cost)
            epochs_classification_lst.append(classification_cost)
            epochs_overall_lst.append(overall_cost)


            print('---------Running validation for student -------------')
            test_loss_student, avg_f1_score_0_student, avg_f1_score_1_student, avg_accuracy_score_student, avg_mcc_score_student = test(student,val_dataloader)

            epochs_f1_score_0_lst_student.append(avg_f1_score_0_student)
            epochs_f1_score_1_lst_student.append(avg_f1_score_1_student)
            epochs_accuracy_lst_student.append(avg_accuracy_score_student)
            epochs_mcc_lst_student.append(avg_mcc_score_student)
            epochs_cost_lst_student.append(test_loss_student)

            writer.add_scalar('mcc_score/Student',avg_mcc_score_student,epoch)
            writer.add_scalar('ValidationLoss/Student',test_loss_student,epoch)


            

            if float("{:.2f}".format(avg_mcc_score_student)) >  best_mcc_student :
                    torch.save(student.state_dict(), f'../models/pseudolabeling/StudentModels/studentModel_{model_name}.bin')
                    best_mcc_student = float("{:.3f}".format(avg_mcc_score_student))

            print('---------Running validation for teacher -------------')

            test_loss_teacher, avg_f1_score_0_teacher, avg_f1_score_1_teacher, avg_accuracy_score_teacher, avg_mcc_score_teacher=test(teacher,val_dataloader)

            epochs_f1_score_0_lst_teacher.append(avg_f1_score_1_teacher)
            epochs_f1_score_1_lst_teacher.append(avg_f1_score_0_teacher)
            epochs_accuracy_lst_teacher.append(avg_accuracy_score_teacher)
            epochs_mcc_lst_teacher.append(avg_mcc_score_teacher)
            epochs_cost_lst_teacher.append(test_loss_teacher)

            writer.add_scalar('mcc_score/Teacher',avg_mcc_score_teacher,epoch)
            writer.add_scalar('ValidationLoss/Teacher',test_loss_teacher,epoch)

            f.write(f"Consistency_loss {epoch+1} : {str(consistency_cost)}" + '\n')
            f.write(f"Classification_loss {epoch+1} : {str(classification_cost)}" + '\n')
            f.write(f"Overall_loss {epoch+1} : {str(overall_cost)}" + '\n')
            f.write(f"'mcc_score/Teacher {epoch+1} : {str(avg_mcc_score_teacher)}" + '\n')
            f.write(f"ValidationLoss/Teacher {epoch+1} : {str(test_loss_teacher)}" + '\n')
            f.write(f"'mcc_score/Student {epoch+1} : {str(avg_mcc_score_student)}" + '\n')
            f.write(f"ValidationLoss/Student {epoch+1} : {str(test_loss_student)}" + '\n')

            

            if float("{:.2f}".format(avg_mcc_score_teacher)) >  best_mcc_teacher :
                    torch.save(teacher.state_dict(), f'../models/pseudolabeling/TeacherModels/teacherModel_{model_name}.bin')
                    best_mcc_teacher = float("{:.3f}".format(avg_mcc_score_teacher))        


            if early_stopping.step(float("{:.2f}".format(avg_mcc_score_teacher))): #mcc score for early stopping "{:.2f}".format(a_float)
                      break 

#             if epoch %5 == 0 :
#                 writer.add_hparams(
#                     { "lr": float(config.lr), 
#                      "consistency_ratio": float(get_consistency_weight(epoch)) , 
#                      "alpha":config.alpha, 
#                      "teacher_noise_gaussian_std":config.gaussian_noise_std_teacher,
#                      "student_noise_gaussian_std":config.gaussian_noise_std_student,
#                      "epoch_current":epoch+1,
#                      "labelled_data": len_labelled_data, 
#                      "unlabelled_data":len_unlabelled_data 

#                     },
#                     {
#                     "mcc_score_teacher": float(avg_mcc_score_teacher),
#                     "loss_teacher": float(test_loss_teacher),
#                     "mcc_score_student" : float(avg_mcc_score_student),
#                     "loss_student" : float(test_loss_student)
#                     },
#                 )

#     writer.flush()  
#     writer.close()    
    return (epochs_consistency_lst,epochs_classification_lst,epochs_overall_lst,epochs_f1_score_0_lst_student,epochs_f1_score_1_lst_student,
           epochs_accuracy_lst_student,epochs_mcc_lst_student,epochs_cost_lst_student,epochs_f1_score_0_lst_teacher,epochs_f1_score_1_lst_teacher,
            epochs_accuracy_lst_teacher,epochs_mcc_lst_teacher,epochs_cost_lst_teacher)

In [569]:
config.threshold

0.8

In [42]:
MeanTeacher_initWithsuptrainedmodel_withConsistencyRampup_for_5_epochs
MeanTeacher_withConsistencyRampup_for_1500_steps
teacherModel_v1_rampup_const_cost_1500_steps_withDropout
v1_initSupModel_rampup_const_cost_5_epochs_withDropout
MeanTeacher_withConsistencyRampup_for_1500_steps_to_1
v1_rampup_const_cost_1500_steps_to_1_withDropout
pseudolabeling_750_label_1250_unlabel_samples_confidence_60_wo_constcost
v1_750_label_1250_unlabel_pseudolabeling_confidence_60_wo_constcost
pseudolabeling_750_label_1250_unlabel_samples_confidence_80
v1_750_label_1250_unlabel_pseudolabeling_confidence_80

NameError: name 'MeanTeacher_initWithsuptrainedmodel_withConsistencyRampup_for_5_epochs' is not defined

In [570]:
writer = SummaryWriter(f"runs/AugSupervised_and_PseudoLabelingTraining/pseudo_label_{label_data_required}_labeled_data_{unlabel_data_required}_unlabel_data_confidence_80_final")

In [46]:
writer.add_text('text', 'Ramup from 0 to 1 in 1200 steps. Batch size was 8 so total steps in 2 epochs were 1200. Also, noise used in teacher is more than student 0.5 and 0.3 respectively for teacher and student models')

In [531]:
exp_name = f'pseudolabeling_{len_labelled_data}_label_{len_unlabelled_data}_unlabel_samples_confidence_{config.threshold}_wo_conscost'
model_name = f'v1_{len_labelled_data}_label_{len_unlabelled_data}_unlabel_pseudolabeling_confidence_{config.threshold}_wo_conscost'
print(exp_name)
print(model_name)

pseudolabeling_1750_label_250_unlabel_samples_confidence_0.6_wo_conscost
v1_1750_label_250_unlabel_pseudolabeling_confidence_0.6_wo_conscost


In [571]:
exp_name = f'pseudolabeling_{len_labelled_data}_label_{len_unlabelled_data}_unlabel_samples_confidence_{config.threshold}_final'
model_name = f'v1_{len_labelled_data}_label_{len_unlabelled_data}_unlabel_pseudolabeling_confidence_{config.threshold}_final'
print(exp_name)
print(model_name)

pseudolabeling_1250_label_750_unlabel_samples_confidence_0.8_final
v1_1250_label_750_unlabel_pseudolabeling_confidence_0.8_final


In [ ]:
early_stopping = EarlyStopping(patience=8)
metrices = train_MeanTeacher(train_dataloader,val_dataloader,len_labelled_data,len_unlabelled_data, exp_name , model_name, early_stopping,writer,with_const_cost =True, dropout_layer=True)

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are ini

Current epoch is 1 of 25


 54%|█████▍    | 51/94 [00:17<00:14,  2.99it/s]

train loss :  0.1155849552154541
Consistency Cost : 0.48893072485923766
Classification Cost : 0.10314843080937862
Consistency Weight is: 0.006737946999085467


  0%|          | 0/24 [00:00<?, ?it/s]

...

---------Running validation for student -------------


  0%|          | 0/24 [00:00<?, ?it/s]

Overall validation loss: tensor(1.5444, device='cuda:0')
Average F1 Validation score for whole sentence class 0 : 0.4338712360672102
Average F1 Validation score for whole sentence class 1 : 0.8310579357593209
Average Accuracy Validation score whole sentence  : 0.7397721189875354
Average mcc Validation score whole sentence : 0.3541615207345221
Classification Report :
               precision    recall  f1-score   support

           0       0.30      0.75      0.43      1734
           1       0.95      0.74      0.83     11343

    accuracy                           0.74     13077
   macro avg       0.63      0.74      0.63     13077
weighted avg       0.87      0.74      0.78     13077

---------Running validation for teacher -------------


100%|██████████| 24/24 [00:01<00:00, 15.20it/s]


Overall validation loss: tensor(1.5215, device='cuda:0')
Average F1 Validation score for whole sentence class 0 : 0.4381815131689314
Average F1 Validation score for whole sentence class 1 : 0.8341504481751101
Average Accuracy Validation score whole sentence  : 0.7439015064617267
Average mcc Validation score whole sentence : 0.3594026738792964
Classification Report :
               precision    recall  f1-score   support

           0       0.31      0.75      0.44      1734
           1       0.95      0.74      0.83     11343

    accuracy                           0.74     13077
   macro avg       0.63      0.75      0.64     13077
weighted avg       0.87      0.74      0.78     13077

terminating because of early stopping!


In [573]:
# model = EntityModel() 
# model.cuda()

model.load_state_dict(torch.load(f'../models/pseudolabeling/StudentModels/studentModel_{model_name}.bin'))
model.eval() # for 1k, 1.0 k data- teacher model -- less seq len
test_metrics = engine.eval_fn(test_dataloader, model)
student_mcc_test = test_metrics[4]# student

# Thesis/src/code/models/pseudolabeling/TeacherModels/.6.bin

  0%|          | 0/125 [00:00<?, ?it/s]/root/Thesis/src/code/code_files/engine.py:409: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  np_arr_labels = np.array(tar_labels)
100%|██████████| 125/125 [00:07<00:00, 15.99it/s]


Average F1 Validation score for whole sentence class 0 : 0.4252788104089219
Average F1 Validation score for whole sentence class 1 : 0.8465854758682906
Average Accuracy Validation score whole sentence  : 0.7578182853887782
Average mcc Validation score whole sentence : 0.3352138147070163
Average F1 Validation score for source sentence class 0 : 0.4078263527973097
Average F1 Validation score for source sentence class 1 : 0.7995653973509934
Average Accuracy Validation score source sentence  : 0.700502512562814
Average mcc Validation score source sentence : 0.2942978208502161
Average F1 Validation score for target sentence class 0 : 0.45156065042676474
Average F1 Validation score for target sentence class 1 : 0.7707731166835925
Average Accuracy Validation score target sentence  : 0.6766812355382524
Average mcc Validation score target sentence : 0.3102254926852018
Average F1 Validation score for gaps in target sentence class 0 : 0.0
Average F1 Validation score for gaps in target sentence cl

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


In [534]:
model_name

'v1_1750_label_250_unlabel_pseudolabeling_confidence_0.6_wo_conscost'

In [574]:

model.load_state_dict(torch.load(f'../models/pseudolabeling/TeacherModels/teacherModel_{model_name}.bin'))
model.eval() # for 1k, 1.0 k data- teacher model -- less seq len
test_metrics = engine.eval_fn(test_dataloader, model)
# Thesis/src/code/models/pseudolabeling/TeacherModels/.bin
teacher_mcc_test = test_metrics[4]

  0%|          | 0/125 [00:00<?, ?it/s]/root/Thesis/src/code/code_files/engine.py:409: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  np_arr_labels = np.array(tar_labels)
100%|██████████| 125/125 [00:07<00:00, 16.04it/s]


Average F1 Validation score for whole sentence class 0 : 0.4258900541046023
Average F1 Validation score for whole sentence class 1 : 0.8444315734221435
Average Accuracy Validation score whole sentence  : 0.7551979493021931
Average mcc Validation score whole sentence : 0.3370006778784989
Average F1 Validation score for source sentence class 0 : 0.40930444006667677
Average F1 Validation score for source sentence class 1 : 0.7977271547921748
Average Accuracy Validation score source sentence  : 0.6986470815616544
Average mcc Validation score source sentence : 0.2971520842612885
Average F1 Validation score for target sentence class 0 : 0.4512247529007306
Average F1 Validation score for target sentence class 1 : 0.7657801650727107
Average Accuracy Validation score target sentence  : 0.6716861938516914
Average mcc Validation score target sentence : 0.3107029903585249
Average F1 Validation score for gaps in target sentence class 0 : 0.0
Average F1 Validation score for gaps in target sentence c

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


In [375]:
model.load_state_dict(torch.load(f'../models/MeanTeacher_models/TeacherModels/teacherModel_v1_with_1250_labeled_750_unlabeled_data_withdropout_alpha_0.99_noises_changed_t_0.5_s_0.3.bin'))
model.eval() # for 1k, 1.0 k data- teacher model -- less seq len
test_metrics = engine.eval_fn(test_dataloader, model)
# Thesis/src/code/models/MeanTeacher_models/TeacherModels/teacherModel_v1_with_1250_labeled_750_unlabeled_data_withdropout_alpha_0.99_noises_changed_t_0.5_s_0.3.binteacher_mcc_test = test_metrics[4]

  0%|          | 0/125 [00:00<?, ?it/s]/root/Thesis/src/code/code_files/engine.py:409: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  np_arr_labels = np.array(tar_labels)
100%|██████████| 125/125 [00:07<00:00, 17.12it/s]


Average F1 Validation score for whole sentence class 0 : 0.4397887777443867
Average F1 Validation score for whole sentence class 1 : 0.8886100369621074
Average Accuracy Validation score whole sentence  : 0.8141697522073483
Average mcc Validation score whole sentence : 0.34438902232277757
Average F1 Validation score for source sentence class 0 : 0.42148843784711704
Average F1 Validation score for source sentence class 1 : 0.8630637952052012
Average Accuracy Validation score source sentence  : 0.7785465790490916
Average mcc Validation score source sentence : 0.3048245354117049
Average F1 Validation score for target sentence class 0 : 0.4694508894044857
Average F1 Validation score for target sentence class 1 : 0.834794335805799
Average Accuracy Validation score target sentence  : 0.7480442208102251
Average mcc Validation score target sentence : 0.33087824908257835
Average F1 Validation score for gaps in target sentence class 0 : 0.0
Average F1 Validation score for gaps in target sentence 

In [575]:
writer.add_hparams(
                    { "lr": float(config.lr),  
                     "labelled_data": len_labelled_data, 
                     "validation_data":len(eval_data_ssl),
                     "alpha":config.alpha, 
                     "teacher_noise_gaussian_std":config.gaussian_noise_std_teacher,
                     "student_noise_gaussian_std":config.gaussian_noise_std_student,
                     "unlabelled_data":len_unlabelled_data, 
                    },
                    {
                    "mcc_score_test" : float(0),
                    "mcc_score_teacher": float(teacher_mcc_test) ,
                    "mcc_score_student" : float(student_mcc_test),
                    },
                )

In [161]:
fail hoga ya pe
metrices = train_MeanTeacher(train_dataloader,val_dataloader,len_labelled_data,len_unlabelled_data,writer, early_stopping,dropout_layer=True, noise_layer=True) # alpha = 0.995 and full data w/o noise and dropout

SyntaxError: invalid syntax (<ipython-input-161-e5dcf95232ff>, line 1)

In [ ]:
consistency_cost_lst = get_items(metrices[0])
classification_cost_lst = get_items(metrices[1])
overall_cost_lst = get_items(metrices[2])
f1_class0_student_test = metrices[3]
f1_class1_student_test= metrices[4]
accuracy_student = metrices[5]
mcc_cost_student = metrices[6]
test_cost_student = tensors_to_lst(metrices[7])
f1_class1_teacher_test=metrices[8]
f1_class0_teacher_test = metrices[9]
accuracy_teacher = metrices[10]
mcc_cost_teacher = metrices[11]
test_cost_teacher=tensors_to_lst(metrices[12])

print('Consistency_lost_train :',consistency_cost_lst,'\n',
      'Classificataion_lost_train : ',classification_cost_lst,'\n',
      'Overall_loss_train :',overall_cost_lst,'\n',
      'f1_class0_student_test :',f1_class0_student_test,'\n',
      'f1_class1_student_test :',f1_class1_student_test,'\n',
      'accuracy_student :',accuracy_student,'\n',
      'mcc_cost_student :',mcc_cost_student,'\n',
      'test_cost_student :',test_cost_student,'\n',
      'f1_class0_teacher_test :',f1_class0_teacher_test,'\n',
      'f1_class1_teacher_test :',f1_class1_teacher_test,'\n',
    'accuracy_teacher :', accuracy_teacher,'\n',
      'mcc_cost_teacher',mcc_cost_teacher,'\n',
      'test_cost_teacher',test_cost_teacher)

In [ ]:
# # model= EntityModel()
# optimizer_parameters_model = list(model.named_parameters())
  
# no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
# optimizer_parameters = [
# {
#     "params": [
#         p for n, p in optimizer_parameters_model if not any(nd in n for nd in no_decay)
#     ],
#     "weight_decay": 0.001,
# },
# {
#     "params": [
#         p for n, p in optimizer_parameters_model if any(nd in n for nd in no_decay)
#     ],
#     "weight_decay": 0.0,
# }]
# optimizer = AdamW(optimizer_parameters, lr=config.lr)
# num_train_steps = int(len(train_dataloader) * config.EPOCHS)
# #     scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
# #         optimizer, T_max=num_train_steps)
# scheduler = get_linear_schedule_with_warmup(
# optimizer, num_warmup_steps=0, num_training_steps=num_train_steps)
# float(scheduler.get_last_lr()[0])

In [ ]:
def get_items(lst):
    new_lst = []
    for items in lst:
        for item in items :
            new_lst.append(float(item))
    return new_lst

In [ ]:
def tensors_to_lst(lst):
    new_lst = []
    for items in lst:
        new_lst.append(items.item())
    return new_lst

In [ ]:
def plot_against_epochs(matrix1,matrix2,label1 , label2 ,y_label , title):
    
    epochs = np.arange(1,config.EPOCHS+1)
    # print(epochs)
    plt.plot(epochs,matrix1,label=label1)
    if matrix2 is not None :
        plt.plot(epochs,matrix2,label=label2)
    plt.xlabel('epochs')
    plt.ylabel(y_label)
    plt.title(title)
    plt.grid()
    plt.legend()
    plt.show()

<h2>Running without Noise embeddings and dropout </h2> -- <h4> No weighted Loss </h4>

In [ ]:
metrices = train_MeanTeacher(train_dataloader,val_dataloader,noise_layer=False) # alpha = 0.995 and full data w/o noise and dropout

consistency_cost_lst = get_items(metrices[0])
classification_cost_lst = get_items(metrices[1])
overall_cost_lst = get_items(metrices[2])
f1_class0_student_test = metrices[3]
f1_class1_student_test= metrices[4]
accuracy_student = metrices[5]
mcc_cost_student = metrices[6]
test_cost_student = tensors_to_lst(metrices[7])
f1_class1_teacher_test=metrices[8]
f1_class0_teacher_test = metrices[9]
accuracy_teacher = metrices[10]
mcc_cost_teacher = metrices[11]
test_cost_teacher=tensors_to_lst(metrices[12])

print('Consistency_lost_train :',consistency_cost_lst,'\n',
      'Classificataion_lost_train : ',classification_cost_lst,'\n',
      'Overall_loss_train :',overall_cost_lst,'\n',
      'f1_class0_student_test :',f1_class0_student_test,'\n',
      'f1_class1_student_test :',f1_class1_student_test,'\n',
      'accuracy_student :',accuracy_student,'\n',
      'mcc_cost_student :',mcc_cost_student,'\n',
      'test_cost_student :',test_cost_student,'\n',
      'f1_class0_teacher_test :',f1_class0_teacher_test,'\n',
      'f1_class1_teacher_test :',f1_class1_teacher_test,'\n',
    'accuracy_teacher :', accuracy_teacher,'\n',
      'mcc_cost_teacher',mcc_cost_teacher,'\n',
      'test_cost_teacher',test_cost_teacher)

In [ ]:
consistency_cost_lst = get_items(metrices[0])
classification_cost_lst = get_items(metrices[1])
overall_cost_lst = get_items(metrices[2])
f1_class0_student_test = metrices[3]
f1_class1_student_test= metrices[4]
accuracy_student = metrices[5]
mcc_cost_student = metrices[6]
test_cost_student = tensors_to_lst(metrices[7])
f1_class1_teacher_test=metrices[8]
f1_class0_teacher_test = metrices[9]
accuracy_teacher = metrices[10]
mcc_cost_teacher = metrices[11]
test_cost_teacher=tensors_to_lst(metrices[12])

print('Consistency_lost_train :',consistency_cost_lst,'\n',
      'Classificataion_lost_train : ',classification_cost_lst,'\n',
      'Overall_loss_train :',overall_cost_lst,'\n',
      'f1_class0_student_test :',f1_class0_student_test,'\n',
      'f1_class1_student_test :',f1_class1_student_test,'\n',
      'accuracy_student :',accuracy_student,'\n',
      'mcc_cost_student :',mcc_cost_student,'\n',
      'test_cost_student :',test_cost_student,'\n',
      'f1_class0_teacher_test :',f1_class0_teacher_test,'\n',
      'f1_class1_teacher_test :',f1_class1_teacher_test,'\n',
    'accuracy_teacher :', accuracy_teacher,'\n',
      'mcc_cost_teacher',mcc_cost_teacher,'\n',
      'test_cost_teacher',test_cost_teacher)

<h3>Plotting</h3>

In [ ]:
plot_against_epochs(test_cost_teacher,test_cost_student,label1 = 'ValidationLoss_teacher', 
                    label2 = 'ValidationLoss_student', y_label = 'loss', title ='Meanteacher_XLMRoberta' )
plot_against_epochs(mcc_cost_teacher,mcc_cost_student,label1 = 'mcc_teacher', 
                    label2 = 'mcc_student', y_label = 'mcc_score_test', title ='Meanteacher_XLMRoberta' )
plot_against_epochs(classification_cost_lst,None,label1 = 'classification_cost_train', 
                    label2 = None, y_label = 'loss', title ='Meanteacher_XLMRoberta' )
plot_against_epochs(consistency_cost_lst,None,label1 = 'Consistency_lost_train', 
                    label2 = None, y_label = 'loss', title ='Meanteacher_XLMRoberta' )

In [ ]:
for name,param in model.named_parameters():
    if 'classifier' not in name:
        param.requires_grad = True

In [ ]:
# writer.add_text('Text','The Labelled data used in this experiment was 3200 and unlabelled used was 6400')


In [ ]:
writer = SummaryWriter("runs/MeanTeacherTraining/alpha_0.995_trail1")

In [ ]:
metrices = train_MeanTeacher(train_dataloader,val_dataloader,len_labelled_data,len_unlabelled_data,writer, noise_layer=True) # alpha = 0.995 and full data w/o noise and dropout

consistency_cost_lst = get_items(metrices[0])
classification_cost_lst = get_items(metrices[1])
overall_cost_lst = get_items(metrices[2])
f1_class0_student_test = metrices[3]
f1_class1_student_test= metrices[4]
accuracy_student = metrices[5]
mcc_cost_student = metrices[6]
test_cost_student = tensors_to_lst(metrices[7])
f1_class1_teacher_test=metrices[8]
f1_class0_teacher_test = metrices[9]
accuracy_teacher = metrices[10]
mcc_cost_teacher = metrices[11]
test_cost_teacher=tensors_to_lst(metrices[12])

print('Consistency_lost_train :',consistency_cost_lst,'\n',
      'Classificataion_lost_train : ',classification_cost_lst,'\n',
      'Overall_loss_train :',overall_cost_lst,'\n',
      'f1_class0_student_test :',f1_class0_student_test,'\n',
      'f1_class1_student_test :',f1_class1_student_test,'\n',
      'accuracy_student :',accuracy_student,'\n',
      'mcc_cost_student :',mcc_cost_student,'\n',
      'test_cost_student :',test_cost_student,'\n',
      'f1_class0_teacher_test :',f1_class0_teacher_test,'\n',
      'f1_class1_teacher_test :',f1_class1_teacher_test,'\n',
    'accuracy_teacher :', accuracy_teacher,'\n',
      'mcc_cost_teacher',mcc_cost_teacher,'\n',
      'test_cost_teacher',test_cost_teacher)

In [ ]:
writer

In [ ]:
plot_against_epochs(test_cost_teacher,test_cost_student,label1 = 'ValidationLoss_teacher', 
                    label2 = 'ValidationLoss_student', y_label = 'loss', title ='Meanteacher_XLMRoberta' )
plot_against_epochs(mcc_cost_teacher,mcc_cost_student,label1 = 'mcc_teacher', 
                    label2 = 'mcc_student', y_label = 'mcc_score_test', title ='Meanteacher_XLMRoberta' )
plot_against_epochs(classification_cost_lst,None,label1 = 'classification_cost_train', 
                    label2 = None, y_label = 'loss', title ='Meanteacher_XLMRoberta' )
plot_against_epochs(consistency_cost_lst,None,label1 = 'Consistency_lost_train', 
                    label2 = None, y_label = 'loss', title ='Meanteacher_XLMRoberta' )